# Archgit - How To Create A Commit From Scratch

Many software developers use Git every day, but have you ever wondered how all of the Git magic acutally works? "Sure it uses hashes and stuff" is what you may be thinking right now. Which is correct by the way but it is only part of the story. I will try to enlighten you with some details about the guts of git with this interactive notebook.

To do so I will explain how to create a basic Git repository and commit a file without using any git commands all. Welcome to Archgit or how to create a commit from scratch! Without further ado, let's get started!

First we need to create a bare bones repository. With `git init` the repository is already bloated with lots of unnecessary stuff...

In [16]:
!git init -q ./bloated
!tree ./bloated/.git/
!rm -rf ./bloated

./bloated/.git/
├── HEAD
├── config
├── description
├── hooks
│   ├── applypatch-msg.sample
│   ├── commit-msg.sample
│   ├── fsmonitor-watchman.sample
│   ├── post-update.sample
│   ├── pre-applypatch.sample
│   ├── pre-commit.sample
│   ├── pre-push.sample
│   ├── pre-rebase.sample
│   ├── pre-receive.sample
│   ├── prepare-commit-msg.sample
│   └── update.sample
├── info
│   └── exclude
├── objects
│   ├── info
│   └── pack
└── refs
    ├── heads
    └── tags

8 directories, 15 files


What we actually only need is the `.git` directory with the `HEAD` file, which tells git from where to start to build our working directory, as well as the `objects` and `refs` direcotries. The `objects` directory is where Git actually stores all the data and the `refs` directory enables us to save references to specific points in time in a human readable or rather rememberable format.

In [23]:
!mkdir -p basicrepo/.git/{refs,objects}
!echo "ref: refs/heads/master" >> ./basicrepo/.git/HEAD
!tree basicrepo/.git
!git --git-dir=$PWD/basicrepo/.git --work-tree=$PWD/basicrepo status

basicrepo/.git
├── HEAD
├── objects
└── refs

2 directories, 1 file
On branch master

No commits yet

nothing to commit (create/copy files and use "git add" to track)


As we can see `git status` is happy with our minimal repository and says that we are on the *master* branch, which is a little bit of a lie at the moment since there is no reference to the *master* branch in `refs/heads`.

Let's takse this opportunity to get into how git actually stores *things* and first and foremost what kind of *things* there actually are

## Things in Git - Blobs, Trees and Commits